# Le modèle de propagation de front de type Nagumo

On considère l'équation de Nagumo :

\begin{equation} 
\frac{\partial y}{\partial t} = D \frac{y}{\partial x^2} + k \, y^2 (1 - y)
\end{equation}

In [ ]:
import numpy as np
import time

from scipy.integrate import solve_ivp

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import column

from mylib.nagumo_model import nagumo_model
import mylib.integration_spatially_extended as integration
from mylib.integration_spatially_extended import parareal

output_notebook(hide_banner=True)

In [ ]:
def compute_integration_time(tini, tend, yini, fcn, fcn_reac, fcn_diff, method, nt, tol, ntimes=100):
        
    t0 = time.time()
    for i in range(ntimes):
        integration.integrate(method, tini, tend, nt, tol, yini, fcn, fcn_reac, fcn_diff)
    t1 = time.time()
    time_average = (t1-t0) / ntimes
    
    return time_average

## Solution obtenue par l'algorithme pararéel 

In [ ]:
def parareal_sol():
    
    k = 1.
    d = 1.

    xmin = -70.
    xmax = 70.
    # nb of points including boundary conditions
    nxib = 1001
    nx = nxib-2
    tini = 0.0
    tend = 50.0
                
    nm = nagumo_model(k=k, d=d, xmin=xmin, xmax=xmax, nx=nx)
    fcn = nm.fcn_radau
    #fcn_diff = nm.fcn_diff_rock
    fcn_diff = nm.fcn_diff_radau
    fcn_reac = nm.fcn_reac_radau
    fcn_exact = nm.fcn_exact    

    dx = (xmax-xmin)/(nxib-1)
    x = np.linspace(xmin+dx, xmax-dx, nx)
    yini = fcn_exact(tini)    
    
    # parareal parameters
    nb_sub = 25
    coarse_method = "strang"
    nc = 5
    tolc = 1.e-6
    fine_method = "strang"
    nf = 50
    tolf = 1.e-6
    max_iter = nb_sub
    eps = 1.e-6
  
    dT = (tend-tini)/nb_sub
    # compute time for coarse integration
    Tg = compute_integration_time(tini, tini+dT, yini, fcn, fcn_reac, fcn_diff, coarse_method, nc, tolc, ntimes=10)
    print(f"Elapsed time (s) for coarse integration : {Tg:.8f}")
    # compute time for fine integration
    Tf = compute_integration_time(tini, tini+dT, yini, fcn, fcn_reac, fcn_diff, fine_method, nf, tolf, ntimes=2)
    ##Tf = compute_integration_time(tini, tend, yini, fcn, fine_method, nb_sub*nf, tolf, ntimes=10)
    print(f"Elapsed time (s) for fine integration   : {Tf:.8f}")
    
    # compute reference solution
    print("\nCompute reference solution ...")
    #sol_ref = integration.radau5(tini, tend, yini, fcn, njac=1, atol=1.e-12, rtol=1.e-12)
    sol_ref = integration.compute_ref_sol(tini, tend, nb_sub, yini, fcn)
    
    # parareal algorithm
    print("\nParareal algorithm\n")     
    sol = parareal(tini, tend, nb_sub, yini, fcn, fcn_reac, fcn_diff, coarse_method, nc, tolc, 
                   fine_method, nf, tolf, max_iter, eps)
    
    print("\nGlobal error:")     
    err = np.linalg.norm(sol.Y-sol_ref.Y)/np.sqrt(nb_sub*nx)
    print(f" || Yi - Yref || = {err:7.2e} (error between parareal and ref solution)")
    
    print("\nEfficiency :")
    print(f"  nb iteration (nit) = {sol.nit}")
    print(f"  nit/n = {sol.nit/nb_sub}")
    print(f"  Tg/Tf = {Tg/Tf}")
    Tpar = (sol.nit/nb_sub) + (sol.nit+1)*(Tg/Tf) 
    print(f"  Tseq/Tpar = {1/Tpar}")

    fig = figure(x_range=(xmin, xmax), plot_height=400, plot_width=950)
    fig.line(x, sol_ref.Y[:,-1], color="grey")
    fig.x(x, sol.Y[:,-1])
    show(fig)

parareal_sol()